In [ ]:
# import warnings
# warnings.filterwarnings("ignore")

# Цель занятия
На этом занятии мы рассмотрим особенности реализаций градиентного бустинга xgboost, lightgbm и catboost.

Мы будем использовать датасет "Bank Marketing". В датасете "Bank Marketing" обычно содержатся данные, связанные с маркетинговыми кампаниями банка. Вот некоторые общие колонки, которые могут присутствовать в датасете "Bank Marketing":

1. **age**: Возраст клиента.

2. **job**: Тип работы клиента.

3. **marital**: Семейное положение клиента.

4. **education**: Образование клиента.

5. **default**: Есть ли у клиента кредитный дефолт (задолженность).

6. **balance**: Баланс на счете клиента.

7. **housing**: Имеет ли клиент ипотеку (жилую недвижимость).

8. **loan**: Имеет ли клиент личный заем.

9. **contact**: Способ связи с клиентом.

10. **day**: День месяца, когда был выполнен последний контакт.

11. **month**: Месяц года, когда был выполнен последний контакт.

12. **duration**: Продолжительность последнего контакта в секундах.

13. **campaign**: Количество контактов, выполненных во время данной кампании.

14. **pdays**: Количество дней, прошедших с момента последнего контакта до предыдущей кампании.

15. **previous**: Количество контактов, выполненных перед текущей кампанией.

16. **poutcome**: Результат предыдущей маркетинговой кампании.

17. **target_variable**: Целевая переменная, которую необходимо предсказать. Например, это может быть информация о том, стал ли клиент подписчиком (1) или нет (0) на определенное предложение банка.

Обратите внимание, что фактические названия колонок и их типы данных могут различаться в зависимости от конкретного датасета "Bank Marketing", поэтому рекомендуется проверить описание колонок в вашем конкретном наборе данных.

In [ ]:
!pip install pandas scikit-learn xgboost lightgbm catboost hyperopt optuna bayesian-optimization

In [ ]:
# Библиотеки общего назначения
import pandas as pd
import time

# Модули метрик
from sklearn.metrics import f1_score

# Модули препроцессинга данных
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Библиотеки градиентного бустинга
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Загрузка данных из CSV файла
url = 'https://datahub.io/machine-learning/bank-marketing/r/bank-marketing.csv'
column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
                'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous',
                'poutcome', 'target_variable']

data = pd.read_csv(url)
data.columns = column_names
data.head()

# Разделение на признаки (X) и целевую переменную (y)
X = data.drop('target_variable', axis=1)
y = data['target_variable'].apply(lambda x: x - 1)


# Разделение на обучающий и тестовый наборы данных
'''Что тут нужно написать?''' = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Определение препроцессора для категориальных и числовых признаков
categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
numerical_transformer = StandardScaler()

# Объединение препроцессоров
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

# Создание пайплайна для препроцессора
preprocessing_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Применение препроцессора к данным
X_train = preprocessing_pipeline.fit_transform(X_train)
X_test = preprocessing_pipeline.transform(X_test)

## XGBoost

XGBoost (eXtreme Gradient Boosting) - это библиотека градиентного бустинга, предназначенная для решения задач классификации и регрессии. Она основана на идее градиентного бустинга, который строит ансамбль слабых моделей (обычно деревьев решений) и объединяет их для получения более сильной и устойчивой модели.

Описание алгоритма XGBoost:

1. Инициализация модели:
   - Инициализируется модель с нулевыми предсказаниями (например, средним значением целевой переменной для задачи регрессии или логарифмом отношения шансов для задачи классификации).

2. Построение базовых моделей (деревьев решений):
   - Каждое дерево строится поэтапно. На каждом этапе добавляется новое дерево с учетом остатков, оставшихся после предыдущих деревьев.
   - Для построения дерева используется критерий информативности, такой как критерий Джини или энтропийный критерий, для определения наилучшего разбиения на каждом узле дерева.
   - Деревья строятся с ограничением на их глубину или другими параметрами, чтобы избежать переобучения.

3. Вычисление градиентов и обновление предсказаний:
   - После построения каждого дерева вычисляются градиенты ошибки между предсказаниями модели и истинными значениями целевой переменной.
   - Предсказания модели обновляются путем добавления взвешенной версии предсказаний нового дерева. Веса определяются скоростью обучения (learning rate), которая контролирует влияние каждого дерева на итоговое предсказание модели.

4. Регуляризация и предотвращение переобучения:
   - Дополнительные механизмы регуляризации в XGBoost помогают предотвратить переобучение и улучшить обобщающую способность модели.
   - XGBoost предлагает несколько методов регуляризации, таких как L1- и L2-регуляризация (также известные как регуляризация Лассо и ридж), которые помогают контролировать сложность модели и предотвращать переобучение. Эти методы добавляют штрафы к функции потерь, которые зависят от весов модели.

6. Функция потерь:
   - XGBoost использует адаптивную функцию потерь, которая сочетает в себе различные функции потерь в зависимости от значения целевой переменной. Например, для задачи классификации с двумя классами может использоваться кросс-энтропия ($L(y, \hat{y}) = -y \log(\hat{y}) - (1-y) \log(1-\hat{y})$), а для задачи регрессии - среднеквадратичная ошибка $(\frac{1}{n}\sum (y - \hat{y})^2)$.
   
7. Ансамблирование деревьев:
   - Поскольку XGBoost строит ансамбль из нескольких деревьев, предсказания каждого дерева складываются, чтобы получить итоговое предсказание модели. Модель объединяет прогнозы всех деревьев с учетом их весов, определенных на основе ошибок и значимости.

8. Прогнозирование:
    - После обучения модели XGBoost можно использовать для прогнозирования на новых данных. Модель принимает входные признаки и возвращает прогнозы для задач классификации или регрессии.

При оптимизации модели XGBoost обычно оптимизируют следующие основные параметры:

1. **max_depth**: Глубина дерева. Этот параметр контролирует, насколько глубоким может быть каждое дерево в ансамбле. Большая глубина может привести к переобучению, поэтому его следует настраивать осторожно.

2. **learning_rate**: Скорость обучения. Этот параметр определяет вклад каждого дерева в ансамбле. Более низкое значение learning_rate требует большего числа деревьев для достижения хорошей производительности, но может улучшить обобщающую способность модели.

3. **n_estimators**: Количество деревьев в ансамбле. Этот параметр указывает, сколько деревьев следует построить. Большее количество деревьев может улучшить производительность модели, но слишком большое число может привести к переобучению.

4. **subsample**: Доля обучающих примеров, используемых для построения каждого дерева. Этот параметр контролирует случайную подвыборку данных для каждого дерева. Значение меньше 1.0 позволяет уменьшить переобучение и повысить устойчивость модели.

5. **colsample_bytree**: Доля признаков, используемых для построения каждого дерева. Этот параметр контролирует случайную подвыборку признаков для каждого дерева. Значение меньше 1.0 помогает уменьшить переобучение и повысить разнообразие деревьев.

6. **colsample_bylevel**: Доля признаков, используемых для построения каждого дерева на каждом уровне. Этот параметр контролирует случайную подвыборку признаков для каждого дерева. Значение меньше 1.0 помогает уменьшить переобучение и повысить разнообразие деревьев.

7. **colsample_bynode**: Доля признаков, используемых для построения каждого дерева в каждом узле разбиения. Этот параметр контролирует случайную подвыборку признаков для каждого дерева. Значение меньше 1.0 помогает уменьшить переобучение и повысить разнообразие деревьев.

8. **gamma**: Минимальное уменьшение функции потерь, необходимое для выполнения расщепления в дереве. Этот параметр контролирует регуляризацию модели. Большее значение gamma помогает предотвратить переобучение за счет увеличения порога для выполнения расщепления.

9. **reg_alpha**: Параметр L1 регуляризации. Он помогает контролировать сложность модели и предотвращать переобучение путем добавления штрафов за большие значения весов.

10. **reg_lambda**: Параметр L2 регуляризации. Он помогает контролировать сложность модели и предотвращать переобучение путем добавления штрафов за большие значения весов.

In [ ]:
# Библиотеки и модули оптимизации
from sklearn.model_selection import GridSearchCV

# Определение модели XGBoost
model = xgb.XGBClassifier()

# Определение сетки параметров для оптимизации
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1],
    'n_estimators': [30]
}

# Создание объекта GridSearchCV для оптимизации параметров
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')

# Обучение модели с использованием GridSearchCV
start_time = time.time()
'''Как обучить grid_search по X_train и y_train?'''
xgb_gridsearch_execution_time = '''Как можно определить время выполнения?'''
print("Время работы xgb_gridsearch_execution_time:", xgb_gridsearch_execution_time)

# Лучшие параметры и значение метрики
print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая F1:", grid_search.best_score_)

# Оценка качества модели на тестовом наборе данных
model = xgb.XGBClassifier(**grid_search.best_params_)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
xgb_gridsearch_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", xgb_gridsearch_f1)

In [ ]:
from hyperopt import fmin, tpe, hp, Trials

# Определение функции для оптимизации
def objective(params):
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return -f1_score(y_test, y_pred)  # Целевая метрика, отрицательная F1-мера

# Определение пространства поиска параметров
space = {
    'max_depth': hp.choice('max_depth', [3, 4, 5]),
    'learning_rate': hp.choice('learning_rate', [0.1, 0.01, 0.001]),
    'n_estimators': hp.choice('n_estimators', [30, 50, 100])
}

# Выполнение оптимизации
trials = Trials()
start_time = time.time()
best = fmin(objective, space, algo=tpe.suggest, max_evals=10, trials=trials)
xgb_hyperopt_execution_time = time.time() - start_time
print("Время работы xgb_hyperopt_execution_time:", xgb_hyperopt_execution_time)

# Лучшие параметры и значение метрики
best_params = {
    'max_depth': [3, 4, 5][best['max_depth']],
    'learning_rate': [0.1, 0.01, 0.001][best['learning_rate']],
    'n_estimators': [30, 50, 100][best['n_estimators']]
}
# Лучшие параметры и значение метрики
best_score = -trials.best_trial['result']['loss']


print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = xgb.XGBClassifier(**best_params)
model.fit(X_train, y_train)

y_pred = '''Как сделать предсказание для model по X_test?'''
xgb_hyperopt_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", xgb_hyperopt_f1)

In [ ]:
import optuna

# Определение функции для оптимизации
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_categorical('n_estimators', [30, 50, 100])
    }

    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    return f1  # Целевая метрика, максимизируем F1-меру

# Создание объекта Study для оптимизации
study = optuna.create_study(direction='maximize')
start_time = time.time()
study.optimize(objective, n_trials=10)
xgb_optuna_execution_time = time.time() - start_time
print("Время работы xgb_optuna_execution_time:", xgb_optuna_execution_time)

# Лучшие параметры и значение метрики
best_params = study.best_params
best_score = study.best_value

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = xgb.XGBClassifier(**best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
xgb_optuna_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", xgb_optuna_f1)

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import f1_score

# Определение функции для оптимизации
def objective(max_depth, learning_rate, n_estimators):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators)
    }
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    return f1

# Определение диапазона значений параметров для оптимизации
pbounds = {
    'max_depth': (3, 5),
    'learning_rate': (0.001, 0.1),
    'n_estimators': (30, 100)
}

# Создание объекта BayesianOptimization для оптимизации
optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Запуск оптимизации
start_time = time.time()
optimizer.maximize(init_points=2, n_iter=10)
xgb_bayes_opt_execution_time = time.time() - start_time
print("Время работы xgb_bayes_opt_execution_time:", xgb_bayes_opt_execution_time)

# Лучшие параметры и значение метрики
best_params = optimizer.max['params']
best_f1 = optimizer.max['target']

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_f1)

# Оценка качества модели на тестовом наборе данных с лучшими параметрами
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])
best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
xgb_bayes_opt_f1 = f1_score(y_test, y_pred)

print("F1-мера на тестовом наборе данных:", xgb_bayes_opt_f1)


## LightGBM

Описание алгоритма LightGBM:

1. Инициализация модели:
   - Инициализируется модель с нулевыми предсказаниями (например, средним значением целевой переменной для задачи регрессии или логарифмом отношения шансов для задачи классификации).

2. Построение базовых моделей (деревьев решений):
   - LightGBM использует алгоритм градиентного бустинга над решающими деревьями.
   - Деревья строятся поэтапно, аналогично XGBoost. На каждом этапе добавляется новое дерево с учетом остатков, оставшихся после предыдущих деревьев.
   - LightGBM использует оптимизированную версию алгоритма градиентного бустинга, которая основана на методе обучения по гистограммам. Это позволяет существенно ускорить процесс построения деревьев.

3. Вычисление градиентов и обновление предсказаний:
   - После построения каждого дерева вычисляются градиенты ошибки между предсказаниями модели и истинными значениями целевой переменной.
   - Предсказания модели обновляются путем добавления взвешенной версии предсказаний нового дерева, аналогично XGBoost.

4. Регуляризация и предотвращение переобучения:
   - LightGBM также предлагает несколько методов регуляризации для предотвращения переобучения модели.
   - Он поддерживает L1-регуляризацию (регуляризацию Лассо) и L2-регуляризацию (регуляризацию ридж), которые добавляют штрафы к функции потерь, аналогично XGBoost.

6. Функция потерь:
   - LightGBM поддерживает различные функции потерь, в зависимости от типа задачи (классификация или регрессия). Для задачи бинарной классификации часто используется логистическая функция потерь, а для задачи регрессии - среднеквадратичная ошибка (MSE), аналогично XGBoost.

7. Ансамблирование деревьев:
   - LightGBM также строит ансамбль из нескольких деревьев, и предсказания каждого дерева складываются для получения итогового предсказания модели.

8. Прогнозирование:
   - После обучения модели LightGBM можно использовать для прогнозирования на новых данных, аналогично XGBoost. Модель принимает входные признаки и возвращает прогнозы для задач классификации или регрессии.

При оптимизации модели XGBoost обычно оптимизируют следующие основные параметры:

1. **max_depth**: Глубина дерева. Этот параметр контролирует, насколько глубоким может быть каждое дерево в ансамбле. Большая глубина может привести к переобучению, поэтому его следует настраивать осторожно.

2. **learning_rate**: Скорость обучения. Этот параметр определяет вклад каждого дерева в ансамбле. Более низкое значение learning_rate требует большего числа деревьев для достижения хорошей производительности, но может улучшить обобщающую способность модели.

3. **n_estimators**: Количество деревьев в ансамбле. Этот параметр указывает, сколько деревьев следует построить. Большее количество деревьев может улучшить производительность модели, но слишком большое число может привести к переобучению.

4. **subsample**: Доля обучающих примеров, используемых для построения каждого дерева. Этот параметр контролирует случайную подвыборку данных для каждого дерева. Значение меньше 1.0 позволяет уменьшить переобучение и повысить устойчивость модели.

5. **colsample_bytree**: Доля признаков, используемых для построения каждого дерева. Этот параметр контролирует случайную подвыборку признаков для каждого дерева. Значение меньше 1.0 помогает уменьшить переобучение и повысить разнообразие деревьев.

6. **min_split_gain**: Минимальное уменьшение функции потерь, необходимое для выполнения расщепления в дереве. Этот параметр контролирует регуляризацию модели. Большее значение min_split_gain помогает предотвратить переобучение за счет увеличения порога для выполнения расщепления.

7. **reg_alpha** и **reg_lambda**: Параметры L1 и L2 регуляризации. Они помогают контролировать сложность модели и предотвращать переобучение путем добавления штрафов за большие значения весов.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Определение модели LightGBM
model = lgb.LGBMClassifier()

# Определение сетки параметров для оптимизации
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.05, 0.01, 0.001],
    'n_estimators': [30, 50, 100]
}

# Создание объекта GridSearchCV для оптимизации параметров
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')

# Обучение модели с использованием GridSearchCV
start_time = time.time()
grid_search.fit(X_train, y_train)
lgb_gridsearch_execution_time = time.time() - start_time
print("Время работы lgb_gridsearch_execution_time:", lgb_gridsearch_execution_time)

# Лучшие параметры и значение метрики
print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая F1:", grid_search.best_score_)

# Оценка качества модели на тестовом наборе данных
model = lgb.LGBMClassifier(**grid_search.best_params_)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
lgb_gridsearch_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", lgb_gridsearch_f1)


In [ ]:
from hyperopt import fmin, tpe, hp, Trials

# Определение функции для оптимизации
def objective(params):
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return -f1_score(y_test, y_pred)  # Целевая метрика, отрицательная F1-мера

# Определение пространства поиска параметров
space = {
    'max_depth': hp.choice('max_depth', [3, 4, 5]),
    'learning_rate': hp.choice('learning_rate', [0.1, 0.01, 0.001]),
    'n_estimators': hp.choice('n_estimators', [30, 50, 100])
}

# Выполнение оптимизации
trials = Trials()
start_time = time.time()
best = fmin(objective, space, algo=tpe.suggest, max_evals=10, trials=trials)
lgb_hyperopt_execution_time = time.time() - start_time
print("Время работы lgb_hyperopt_execution_time:", lgb_hyperopt_execution_time)

# Лучшие параметры и значение метрики
best_params = {
    'max_depth': [3, 4, 5][best['max_depth']],
    'learning_rate': [0.1, 0.01, 0.001][best['learning_rate']],
    'n_estimators': [30, 50, 100][best['n_estimators']]
}
best_score = -trials.best_trial['result']['loss']
print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = lgb.LGBMClassifier(**best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
lgb_hyperopt_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", lgb_hyperopt_f1)


In [ ]:
import optuna
import lightgbm as lgb
from sklearn.metrics import f1_score

# Определение функции для оптимизации
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_categorical('n_estimators', [30, 50, 100])
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    return f1  # Целевая метрика, максимизируем F1-меру

# Создание объекта Study для оптимизации
study = optuna.create_study(direction='maximize')
start_time = time.time()
study.optimize(objective, n_trials=10)
lgb_optuna_execution_time = time.time() - start_time
print("Время работы lgb_optuna_execution_time:", lgb_optuna_execution_time)

# Лучшие параметры и значение метрики
best_params = study.best_params
best_score = study.best_value

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = lgb.LGBMClassifier(**best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
lgb_optuna_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", lgb_optuna_f1)


In [ ]:
from bayes_opt import BayesianOptimization

# Определение функции для оптимизации
def objective(max_depth, learning_rate, n_estimators):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators)
    }
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    return f1

# Определение диапазона значений параметров для оптимизации
pbounds = {
    'max_depth': (3, 5),
    'learning_rate': (0.001, 0.1),
    'n_estimators': (30, 100)
}

# Создание объекта BayesianOptimization для оптимизации
optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Запуск оптимизации
start_time = time.time()
optimizer.maximize(init_points=2, n_iter=10)
lgb_bayes_opt_execution_time = time.time() - start_time
print("Время работы lgb_bayes_opt_execution_time:", lgb_bayes_opt_execution_time)

# Лучшие параметры и значение метрики
best_params = optimizer.max['params']
best_f1 = optimizer.max['target']

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_f1)

# Оценка качества модели на тестовом наборе данных с лучшими параметрами
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
lgb_bayes_opt_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", lgb_bayes_opt_f1)

## Catboost

Описание алгоритма CatBoost:

1. Инициализация модели:
   - Инициализируется модель CatBoost с нулевыми предсказаниями (например, средним значением целевой переменной для задачи регрессии или логарифмом отношения шансов для задачи классификации).

2. Построение базовых моделей (деревьев решений):
   - CatBoost использует алгоритм градиентного бустинга над решающими деревьями.
   - Деревья строятся поэтапно, аналогично XGBoost и LightGBM. На каждом этапе добавляется новое дерево с учетом остатков, оставшихся после предыдущих деревьев.
   - CatBoost применяет особый подход к кодированию категориальных признаков, называемый симметричным бинарным кодированием, который учитывает взаимодействия между категориями и признаками.

3. Вычисление градиентов и обновление предсказаний:
   - После построения каждого дерева вычисляются градиенты ошибки между предсказаниями модели и истинными значениями целевой переменной.
   - Предсказания модели обновляются путем добавления взвешенной версии предсказаний нового дерева, аналогично XGBoost и LightGBM.

4. Регуляризация и предотвращение переобучения:
   - CatBoost также предлагает несколько методов регуляризации для предотвращения переобучения модели.
   - Он поддерживает L1-регуляризацию (регуляризацию Лассо) и L2-регуляризацию (регуляризацию ридж), которые добавляют штрафы к функции потерь, аналогично XGBoost и LightGBM.

6. Функция потерь:
   - CatBoost поддерживает различные функции потерь, в зависимости от типа задачи (классификация или регрессия). Для задачи бинарной классификации часто используется логистическая функция потерь, а для задачи регрессии - среднеквадратичная ошибка (MSE), аналогично XGBoost и LightGBM.

7. Ансамблирование деревьев:
   - CatBoost также строит ансамбль из нескольких деревьев, и предсказания каждого дерева складываются, чтобы получить итоговое предсказание модели.
   - CatBoost также учитывает веса деревьев на основе их ошибок и значимости, аналогично XGBoost и LightGBM.

8. Прогнозирование:
   - После обучения модели CatBoost можно использовать для прогнозирования на новых данных. Модель принимает входные признаки и возвращает прогнозы для задач классификации или регрессии.

При оптимизации модели CatBoost можно обратить внимание на следующие важные параметры:

1. **depth**: Глубина дерева. Этот параметр определяет, насколько глубоким может быть каждое дерево. Большая глубина может привести к переобучению, поэтому его следует настраивать осторожно.

2. **learning_rate**: Скорость обучения. Этот параметр контролирует вклад каждого дерева в ансамбле. Более низкое значение learning_rate требует большего числа деревьев для достижения хорошей производительности, но может улучшить обобщающую способность модели.

3. **iterations**: Количество деревьев в ансамбле. Этот параметр указывает, сколько деревьев следует построить. Большее количество деревьев может улучшить производительность модели, но слишком большое число может привести к переобучению.

4. **subsample**: Доля обучающих примеров, используемых для построения каждого дерева. Этот параметр контролирует случайную подвыборку данных для каждого дерева. Значение меньше 1.0 позволяет уменьшить переобучение и повысить устойчивость модели.

5. **colsample_bylevel**: Доля признаков, используемых для построения каждого уровня дерева. Этот параметр контролирует случайную подвыборку признаков для каждого уровня дерева. Значение меньше 1.0 помогает уменьшить переобучение и повысить разнообразие деревьев.

6. **l2_leaf_reg**: L2 регуляризация. Этот параметр помогает контролировать сложность модели и предотвращать переобучение путем добавления штрафов за большие значения весов.

7. **random_strength**: Сила случайности для ускорения обучения. Этот параметр контролирует уровень случайности в выборе признаков при построении дерева. Значение больше 0 добавляет случайность, что может помочь предотвратить переобучение.

8. **bagging_temperature**: Температура бэггинга. Этот параметр контролирует степень случайности в выборе объектов для обучения каждого дерева. Значение больше 0 добавляет случайность, что может помочь предотвратить переобучение.

Важно отметить, что конкретные параметры и их значения могут варьироваться в зависимости от конкретной задачи и набора данных. Рекомендуется провести эксперименты и настраивать параметры с использованием кросс-валидации для достижения наилучшей производительности модели.

In [ ]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

# Определение модели CatBoost
model = CatBoostClassifier(silent=True)

# Определение сетки параметров для оптимизации
# param_grid = {
#     'depth': [3, 4, 5],
#     'learning_rate': [0.1, 0.05, 0.01],
#     'n_estimators': [30, 50, 100]
# }
param_grid = {
    'depth': [5],
    'learning_rate': [0.1, 0.05, 0.01],
    'n_estimators': [30, 50, 100]
}

# Создание объекта GridSearchCV для оптимизации параметров
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')

# Обучение модели с использованием GridSearchCV
start_time = time.time()
grid_search.fit(X_train, y_train)
catboost_gridsearch_execution_time = time.time() - start_time
print("Время работы catboost_gridsearch_execution_time:", catboost_gridsearch_execution_time)

# Лучшие параметры и значение метрики
print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая F1:", grid_search.best_score_)

# Оценка качества модели на тестовом наборе данных
model = CatBoostClassifier(silent=True, **grid_search.best_params_)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
catboost_gridsearch_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", catboost_gridsearch_f1)


In [ ]:
from hyperopt import fmin, tpe, hp, Trials
from sklearn.metrics import f1_score

# Определение функции для оптимизации
def objective(params):
    model = CatBoostClassifier(silent=True, **params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return -f1_score(y_test, y_pred)  # Целевая метрика, отрицательная F1-мера

# Определение пространства поиска параметров
space = {
    'depth': hp.choice('depth', [3, 4, 5]),
    'learning_rate': hp.choice('learning_rate', [0.1, 0.01, 0.001]),
    'iterations': hp.choice('iterations', [30, 50, 100])
}

# Выполнение оптимизации
trials = Trials()
start_time = time.time()
best = fmin(objective, space, algo=tpe.suggest, max_evals=10, trials=trials)
catboost_hyperopt_execution_time = time.time() - start_time
print("Время работы catboost_hyperopt_execution_time:", catboost_hyperopt_execution_time)

# Лучшие параметры и значение метрики
best_params = {
    'depth': [3, 4, 5][best['depth']],
    'learning_rate': [0.1, 0.01, 0.001][best['learning_rate']],
    'iterations': [30, 50, 100][best['iterations']]
}
best_score = -trials.best_trial['result']['loss']

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = CatBoostClassifier(silent=True, **best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
catboost_hyperopt_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", catboost_hyperopt_f1)


In [ ]:
import optuna

# Определение функции для оптимизации
def objective(trial):
    params = {
        'depth': trial.suggest_int('depth', 3, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'iterations': trial.suggest_categorical('iterations', [30, 50, 100])
    }

    model = CatBoostClassifier(silent=True, **params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    return f1  # Целевая метрика, максимизируем F1-меру

# Создание объекта Study для оптимизации
study = optuna.create_study(direction='maximize')
start_time = time.time()
study.optimize(objective, n_trials=10)
catboost_optuna_execution_time = time.time() - start_time
print("Время работы catboost_hyperopt_execution_time:", catboost_optuna_execution_time)

# Лучшие параметры и значение метрики
best_params = study.best_params
best_score = study.best_value

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = CatBoostClassifier(silent=True, **best_params)
model.fit(X_train, y_train)
catboost_optuna_f1 = f1_score(y_test, model.predict(X_test))
print("F1 на тестовом наборе данных:", catboost_optuna_f1)


In [ ]:
from catboost import CatBoostClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import f1_score

# Определение функции для оптимизации
def objective(max_depth, learning_rate, n_estimators):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators)
    }
    model = CatBoostClassifier(silent=True, **params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    return f1

# Определение диапазона значений параметров для оптимизации
pbounds = {
    'max_depth': (3, 5),
    'learning_rate': (0.001, 0.1),
    'n_estimators': (30, 100)
}

# Создание объекта BayesianOptimization для оптимизации
optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Запуск оптимизации
start_time = time.time()
optimizer.maximize(init_points=2, n_iter=10)
catboost_bayes_opt_execution_time = time.time() - start_time
print("Время работы catboost_bayes_opt_execution_time:", catboost_bayes_opt_execution_time)


# Лучшие параметры и значение метрики
best_params = optimizer.max['params']
best_f1 = optimizer.max['target']

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_f1)

# Оценка качества модели с лучшими параметрами на тестовом наборе данных
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])
best_model = CatBoostClassifier(silent=True, **best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
catboost_bayes_opt_f1 = f1_score(y_test, y_pred)
print("F1 на тестовом наборе данных:", catboost_bayes_opt_f1)

In [ ]:
# Создание данных для таблицы сравнения по f1_score и времени работы
params_match = [
    ['XGBoost gridsearch', f'{xgb_gridsearch_f1}', f'{xgb_gridsearch_execution_time}'],
    ['XGBoost hyperopt', f'{xgb_hyperopt_f1}', f'{xgb_hyperopt_execution_time}'],
    ['XGBoost optuna', f'{xgb_optuna_f1}', f'{xgb_optuna_execution_time}'],
    ['XGBoost bayes_opt', f'{xgb_bayes_opt_f1}', f'{xgb_bayes_opt_execution_time}'],
    ['LightGBM gridsearch', f'{lgb_gridsearch_f1}', f'{lgb_gridsearch_execution_time}'],
    ['LightGBM hyperopt', f'{lgb_hyperopt_f1}', f'{lgb_hyperopt_execution_time}'],
    ['LightGBM optuna', f'{lgb_optuna_f1}', f'{lgb_optuna_execution_time}'],
    ['LightGBM bayes_opt', f'{lgb_bayes_opt_f1}', f'{lgb_bayes_opt_execution_time}'],
    ['Catboost gridsearch', f'{catboost_gridsearch_f1}', f'{catboost_gridsearch_execution_time}'],
    ['Catboost hyperopt', f'{catboost_hyperopt_f1}', f'{catboost_hyperopt_execution_time}'],
    ['Catboost optuna', f'{catboost_optuna_f1}', f'{catboost_optuna_execution_time}'],
    ['Catboost bayes_opt', f'{catboost_bayes_opt_f1}', f'{catboost_bayes_opt_execution_time}'],

]

# Создание DataFrame из данных
df = pd.DataFrame(params_match, columns=['Модель и оптимайзер', 'F1', 'Время'])
pd.set_option('display.max_colwidth', None)
# Отображение DataFrame
df.head(len(params_match))

In [ ]:
# Создание данных для таблицы
params_match = [
    ['Скорость обучения (Learning Rate)', 'eta (learning_rate)', 'learning_rate', 'learning_rate'],
    ['Количество деревьев (Number of Trees)', 'n_estimators', 'n_estimators', 'iterations'],
    ['Максимальная глубина дерева (Maximum Depth)', 'max_depth', 'max_depth', 'max_depth'],
    ['Минимальный вес листа дерева, Минимальное количество объектов в листе (Minimum Child Weight)', 'min_child_weight', 'min_child_samples', 'min_data_in_leaf'],
    ['Гамма (Gamma)', 'gamma', 'min_split_gain', 'l2_leaf_reg'],
    ['Доля выборки объектов (Subsample Ratio)', 'subsample', 'subsample', 'subsample'],
    ['Доля выборки признаков (Column Subsampling Ratio)', 'colsample_bytree', 'colsample_bytree', 'colsample_bylevel'],
    ['Лямбда (L2-регуляризация)', 'reg_lambda', 'reg_lambda', 'reg_lambda'],
    ['Альфа (L1-регуляризация)', 'reg_alpha', 'reg_alpha', 'reg_alpha'],
    ['Максимальное изменение шага (Maximum Delta Step)', 'max_delta_step', '-', '-'],
    ['Максимальное количество корзин (Maximum Bin Count)', 'max_bin', 'max_bin', 'max_bin'],
    ['Частота сэмплирования объектов (Sampling Frequency for LGBM)', '-', 'subsample_freq', '-'],
    ['Масштабирование веса положительного класса (Scale the Weight of Positive Class)', 'scale_pos_weight', 'scale_pos_weight', 'scale_pos_weight'],
    ['Режим обучения с помощью градиентного спуска (Booster Type)', 'booster', 'boosting_type', 'boosting_type'],
    ['Коэффициент сжатия (Subsample)', 'subsample', 'subsample', 'subsample'],
    ['Размер выборки для построения дерева (Bagging Fraction)', 'colsample_bytree', 'colsample_bytree', 'colsample_bylevel'],
    ['Способ выбора ветвления (Split Criterion)', 'tree_method', 'tree_learner', 'grow_policy'],
    ['Функция потерь (Loss Function)', 'objective', 'objective', 'loss_function'],
    ['Максимальное количество листьев (Max Leaf Nodes)', '-', 'num_leaves', 'max_leaves']
]

# Создание DataFrame из данных
df = pd.DataFrame(params_match, columns=['Описание параметра', 'XGBoost', 'LightGBM', 'CatBoost'])
pd.set_option('display.max_colwidth', None)
# Отображение DataFrame
df
